<a href="https://colab.research.google.com/github/TollanBerhanu/Semantic-search-on-Slack/blob/main/slack_semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading the dataset

In [1]:
!pip install -q datasets loralib sentencepiece
!pip uninstall transformers
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.3 MB/s eta 0:00:00
     ━━

In [4]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig
import textwrap


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [5]:
import os
import pandas as pd

In [6]:
slack_data = '/content/drive/MyDrive/Colab Notebooks/dataset/slack-data/'
# cwd = os.path.join (os.getcwd(), slack_data)  # join with current_working_directory just in case

df = pd.read_json(slack_data + 'channels.json')

channel_ids = [id for id in df['id']]
channel_names = [ name for name in df['name']]

channels = pd.DataFrame({ 'channel_id': channel_ids, 'channel_name': channel_names } )
channels

,channel_id,channel_name
0,C05D1SE01B7,random
1,C05D77W3N76,general
2,C05D7863DRA,test


In [7]:
# import necessary libraries
import glob
import json

# Return the metadata of each message in the channel
def extract_channel_data(channel_name):
  # use glob to get all the json files in the folder
  daily_json_files = glob.glob(slack_data + channel_name +'/*.json')

  # just return if the channel doesn't exist (or hasn't been exported yet)
  if not daily_json_files:
    return

  metadata = pd.DataFrame(columns = ['message', 'channel', 'date', 'time', 'user_id', 'user_name'])

  # loop over the list of json files (each json file includes every post in that channel for a single day)
  for f in daily_json_files:
    # read the json file
    # today_data = pd.read_json(f)
    with open(f, 'r') as file:
        # Read the contents
        data = file.read()
        # Parse the JSON data
        today_data = json.loads(data)

    today_date = f.split("/")[-1]  # 'f' is the full file path and file name
    print('Extracting...', today_date) # the file name is the date

    # iterate through all the messages of the day
    for msg_data in today_data:
      # Skip if its a "channel_join" type message or if the actual message content is empty
      if ('subtype' in msg_data) or (msg_data['text'] == "") or (msg_data['type'] != 'message'):
        continue
        # TODO: filter out any links, stickers, and other junk
        # TODO: replace @Member references by their real names

      metadata.loc[len(metadata)] = {
            'message': msg_data['text'],
            'channel': channel_name,
            'date': today_date.split(".json")[0], # omit the file extension '.json'
            'time': msg_data['ts'],
            'user_id': msg_data['user'],
            'user_name': msg_data['user_profile']['first_name'] # We can also use 'real_name' if we wanted the full name of the user
      }

  return metadata

In [8]:
extract_channel_data('general').to_json(orient="records")

Extracting... 2023-06-19.json


'[{"message":"hello","channel":"general","date":"2023-06-19","time":"1687166197.580079","user_id":"U05DHDPL4FK","user_name":"kenenisaalemayhu0"},{"message":"<https:\\/\\/haystack.deepset.ai\\/tutorials\\/08_preprocessing>","channel":"general","date":"2023-06-19","time":"1687166202.864639","user_id":"U05DHDPL4FK","user_name":"kenenisaalemayhu0"},{"message":"Good work.. now we don\'t have to worry about exporting data.","channel":"general","date":"2023-06-19","time":"1687166814.786429","user_id":"U05CQ93C3FZ","user_name":"Tollan"},{"message":"It\'s best if we just post random topics here to test the semantic search.","channel":"general","date":"2023-06-19","time":"1687166901.338569","user_id":"U05CQ93C3FZ","user_name":"Tollan"},{"message":"yeah then we\\u2019ll see how we can clean the data","channel":"general","date":"2023-06-19","time":"1687167171.439409","user_id":"U05DHDPL4FK","user_name":"kenenisaalemayhu0"},{"message":"but we can\\u2019t use it for search since it\\u2019ll be very 

In [9]:
# # Cast the values of the column 'Content' into strings
# df['Content'] = df['Content'].astype(str)

# # Join the string values of all the rows in 'Content' into one large corpus of text
# conversations = ' '.join(df['Content'])

Embedding

Generating embedding using sentence transformers

In [10]:
!pip install --upgrade langchain  -q
!pip install sentence_transformers > /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.4 MB/s eta 0:00:00


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

In [12]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# ... is equivalent to ...
# SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [13]:
# Return a list of embeddings for all messages of the channel
def embed_channel_messages(channel_data):
  msg_list = channel_data['message'].astype(str).tolist()
  return embedding_model.embed_documents(msg_list)

In [14]:
los = extract_channel_data('general') # ['message'].tolist()
embed_channel_messages(los)

Extracting... 2023-06-19.json


[[-0.06277172267436981,
  0.054958775639534,
  0.05216480791568756,
  0.08578997105360031,
  -0.0827488899230957,
  -0.07457297295331955,
  0.06855466961860657,
  0.01839641109108925,
  -0.08201140910387039,
  -0.0373847521841526,
  0.012124967761337757,
  0.0035183506552129984,
  -0.004134300164878368,
  -0.04378444701433182,
  0.02180732600390911,
  -0.005102760624140501,
  0.019546672701835632,
  -0.042348768562078476,
  -0.11035966128110886,
  0.005424485541880131,
  -0.05573485791683197,
  0.028052445501089096,
  -0.02315870299935341,
  0.028481420129537582,
  -0.05370955914258957,
  -0.052601657807826996,
  0.03393920138478279,
  0.045388657599687576,
  0.02371852472424507,
  -0.07312081754207611,
  0.05477774515748024,
  0.017047274857759476,
  0.08136042952537537,
  -0.002862693974748254,
  0.011958115734159946,
  0.07355857640504837,
  -0.09423750638961792,
  -0.08136200904846191,
  0.04001539200544357,
  0.000692102184984833,
  -0.013393350876867771,
  -0.05453803762793541,
 

Storing the embeddings in a vector database (Pinecone)

In [15]:
pip install pinecone-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 24.6 MB/s eta 0:00:00


In [16]:
import getpass    # To prompt the user for a password without echoing.
# from langchain.vectorstores import Pinecone

# PINECONE_API_KEY = getpass.getpass("Your API key: ")  # 2b975b00-9ceb-4d2f-b38d-ba6ccb6e532a
# PINECONE_ENV = getpass.getpass("Your env't name: ")   # us-west1-gcp-free

PINECONE_API_KEY = "3c7987b8-6c6b-46a3-8adb-667f21b05421"
PINECONE_ENV = "us-west1-gcp-free"

In [19]:
import pinecone
from langchain.vectorstores import Pinecone

# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_ENV,  # next to api key in console
)

# Make a sample embedding to determine the index dimension
sample_embedding = embedding_model.embed_query('Sample text')

# all_indices = pinecone.list_indexes() # List all the indexed in our pinecone workspace
index_name = "slack-embeddings"
index_dimension = len(sample_embedding)


In [13]:
# Create a pinecone index
print('Creating an index of dimension "'+ str(index_dimension) +'" ...')
pinecone.create_index(index_name, index_dimension)

pinecone.describe_index(index_name)
print('Pinecone index created!')

Creating an index of dimension "384" ...


ApiException: ignored

In [20]:
# Connect to the index
index = pinecone.Index(index_name)
# Current index statistics
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'first-upsert': {'vector_count': 19}},
 'total_vector_count': 19}

In [21]:
# The total number of embedded chunks (if this exceeds 1000, it must be upserted step by step because pinecone can't handle it)
# no_embeddings = len(chunks)

# This will be the size of the batch of vectors sent to pinecone at a time
step = 100

In [22]:
def upsert_channel_embeddings(channel_name, channel_embeddings, channel_data):

  # channel_name = 'general'
  # channel_data = extract_channel_data(channel_name)
  # channel_embeddings = embed_channel_messages(channel_data)

  no_embeddings = len(channel_embeddings)
  print(channel_embeddings)

  parsed_channel_data = json.loads(channel_data.to_json(orient="records"))

  for start in range(0, no_embeddings, step):
    # The end location of the current batch
    end = min(no_embeddings, start+step)    # If it reached the last batch, the end should be the total amount of vectors
                                            # [0..99], [100..199], ... , [1600..1678]  (The last batch should end at 1678)

    # create IDs for all embedded chunks (vectors) ... [channelname_0 -> ... -> channelname_..]
    ids = [channel_name+str(x) for x in range(start, end)]

    # create a records list of current batch for upsert
    records = zip(ids, channel_embeddings[start:end], parsed_channel_data[start:end]) # "vector_id" , [embeddings], {metadata}

    # upsert to Pinecone
      # vectors = [ ( "id1", [0.1,0.2,..], {metadata1} )  ,  ( "id2", [0.4,0.6,..], {metadata2} )  , .. ]
      # namespace = "my-namespace"
    index.upsert(vectors=records, namespace="first-upsert")

    # index stat after current batch upsert
    print('Batch no. ' + str(int( start/step + 1 )) )
    index.describe_index_stats()

  # index stats after all upsert batch
  print('Completed upserting all batches in '+channel_name+' ... ')
  index.describe_index_stats()

In [47]:
# Combine them together
channel_names = channels['channel_name'].tolist()

for channel_name in channel_names:

  channel_data = extract_channel_data(channel_name)

  channel_embeddings = embed_channel_messages(channel_data)

  upsert_channel_embeddings(channel_name, channel_embeddings, channel_data)

Extracting... 2023-06-19.json
[[-0.07584918290376663, -0.021358055993914604, -0.047360558062791824, 0.024587076157331467, -0.05030532553792, -0.07117699831724167, 0.10254044830799103, -0.06500173360109329, 0.08284778147935867, -0.022649534046649933, 0.06542233377695084, -0.05668095499277115, 0.021410761401057243, -0.020286956802010536, -0.04972054436802864, -0.021618368104100227, -0.028856726363301277, -0.039371080696582794, -0.09512753784656525, 0.006624280475080013, -0.05642743408679962, -0.09569098800420761, 0.0026799780316650867, 0.01117519661784172, -0.0966227799654007, -0.0653490200638771, 0.05503646284341812, 0.03184431418776512, 0.040415842086076736, -0.111893430352211, 0.07045644521713257, 0.1069994568824768, 0.01636463589966297, -0.06084941700100899, -0.013645083643496037, -0.00085640192264691, -0.04624719172716141, -0.05976218730211258, -0.0004942309460602701, 0.04147835075855255, 0.0014390397118404508, -0.05436088517308235, 0.05043290555477142, -0.029950425028800964, 0.0157

In [28]:
query = "What shouldn't we worry about?"

embedded_query = embedding_model.embed_query(query)
# embedded_query = embedding_model.embed_query("random stuff")

query_response = index.query(
    namespace="first-upsert",
    top_k=5,
    include_values=False,
    include_metadata=True,
    vector=embedded_query,

    filter={
        "channel": {"$in": ["general", "random"]}
        # "user_id": {"$in": ["U05D1SQDNSH", "U05DHDPL4FK", "U05CQ93C3FZ", "U05D4M7RGQ3"]}
    }
)

context = ""
for ctx in query_response['matches']:
  context += ctx['metadata']['user_name'] + ' said ' + ctx['metadata']['message'] + '\n'

print(context)

Tollan said Good work.. now we don't have to worry about exporting data.
kenenisaalemayhu0 said yeah then we’ll see how we can clean the data
Tollan said And we should also post some stickers... :grinning: :smile: :grin:
kenenisaalemayhu0 said but we can’t use it for search since it’ll be very small
kenenisaalemayhu0 said Random shiii



In [24]:
# Load the model
tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")

model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "samwit/alpaca7B-lora")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [29]:
# Define a function that runs the model
def alpaca_talk(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.2,
    )
    print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )
    for s in generation_output.sequences:
        print(tokenizer.decode(s))

In [30]:
# Provide the model with some instructions along with the context and query
query_and_context = '''
Below is sequence of chat messages related to a certain topic. Write a response that answers the question below based on
what is discussed in the messages. Do not mention anything outside of what is discussed below. If there isn't enough
context, simply reply "This topic was not discussed previously"

### Messages:
{context}

### Question:
{query}

### Response:
'''.format(context=context, query=query)

In [31]:
# Watch the magic happen... it maybe not so magical as you'd expect

# if score < 0.4:
#   print(query_and_context + 'This topic was not discussed previously')
# else
alpaca_talk(query_and_context)

Generating...
 
Below is sequence of chat messages related to a certain topic. Write a response that answers the question below based on
what is discussed in the messages. Do not mention anything outside of what is discussed below. If there isn't enough
context, simply reply "This topic was not discussed previously"

### Messages:
Tollan said Good work.. now we don't have to worry about exporting data.
kenenisaalemayhu0 said yeah then we’ll see how we can clean the data
Tollan said And we should also post some stickers... :grinning: :smile: :grin:
kenenisaalemayhu0 said but we can’t use it for search since it’ll be very small
kenenisaalemayhu0 said Random shiii


### Question:
What shouldn't we worry about?

### Response:
We shouldn't worry about exporting our data because Tollan has already taken care of this issue and we will no longer need to do so. We should however still make sure that all of our data is properly formatted before uploading it into the database.
